<a href="https://colab.research.google.com/github/Rahulyadav2310/PyTorch_learningBook/blob/main/CNN_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Required **Libraries**

In [6]:
# Core PyTorch library
import torch

# Neural network modules
import torch.nn as nn

# Optimizers
import torch.optim as optim

# Dataset and image utilities
from torchvision import datasets, transforms

# DataLoader for batching data
from torch.utils.data import DataLoader

# Check PyTorch version and GPU
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())


2.9.0+cpu
CUDA available: False


Image Transformations

In [7]:
# Convert images to tensors
transform = transforms.Compose([
    transforms.ToTensor()
])


Load MNIST Dataset

In [8]:
# Download and load training dataset
train_dataset = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

# Create DataLoader (batching + shuffling)
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)


Define CNN Model

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolution layer 1
        self.conv1 = nn.Conv2d(
            in_channels=1,   # grayscale image
            out_channels=16, # number of filters
            kernel_size=3    # 3x3 filter
        )

        # Pooling layer
        self.pool = nn.MaxPool2d(2)

        # Fully connected layer
        self.fc = nn.Linear(16 * 13 * 13, 10)

    def forward(self, x):
        # Apply convolution + activation
        x = torch.relu(self.conv1(x))         #ReLU removes negative values
                                             # from the output, allowing the network
                                             # to learn non-linear patterns and train faster.

        # Apply pooling
        x = self.pool(x)

        # Flatten image into vector
        x = torch.flatten(x, 1)

        # Final output layer
        return self.fc(x)


Create Model, Loss, Optimizer

In [10]:
# Create model object
model = CNN()

# Loss function for classification
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


Training Loop (MOST IMPORTANT)

In [11]:
# Train model
for epoch in range(3):
    for images, labels in train_loader:

        # Forward pass
        predictions = model(images)

        # Calculate loss
        loss = loss_fn(predictions, labels)

        # Clear previous gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

    print(f"Epoch {epoch+1} completed, Loss: {loss.item():.4f}")


Epoch 1 completed, Loss: 0.0508
Epoch 2 completed, Loss: 0.0506
Epoch 3 completed, Loss: 0.2287


Model Evaluation (Prediction)

In [12]:
# Switch model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    sample_image, sample_label = train_dataset[0]
    output = model(sample_image.unsqueeze(0))
    prediction = torch.argmax(output)

    print("Predicted digit:", prediction.item())
    print("Actual digit:", sample_label)


Predicted digit: 5
Actual digit: 5
